# Lab 2: Data Acquisition with Bloomberg Terminal

Professional-Grade Data Pipelines in the Financial Innovation Lab

> **Bloomberg Terminal Lab**
>
> This lab is designed for use **in the Financial Innovation Lab** with
> Bloomberg Terminal access. You’ll work with professional-grade data
> and compare it to free API sources.
>
> **Prerequisites:**
>
> -   Bloomberg Terminal software running
> -   Logged into Bloomberg
> -   `bbg-edu` educational package (includes `blpapi` wrapper)
> -   Anaconda/Spyder recommended
>
> **Installation:**
>
> ``` bash
> pip install git+https://github.com/quinfer/bloomberg-edu.git
> ```

## Before You Start: The Big Picture

Welcome to professional-grade financial data. The Bloomberg Terminal is
the industry standard—used by traders, analysts, and portfolio managers
globally. This lab lets you compare professional tools with free APIs
and understand why financial firms pay \$24,000/year per terminal.

> **The Professional Data Advantage**
>
> **Why Bloomberg costs 100x more than free APIs:**
>
> 1.  **Data Quality** → Verified, cleaned, adjusted for corporate
>     actions in real-time
> 2.  **Coverage** → 350+ million securities, 4.5 million issuers,
>     historical data back to 1970s
> 3.  **Speed** → Real-time data, not 15-minute delayed like free
>     sources
> 4.  **Integration** → News, analytics, chat, execution all in one
>     platform
> 5.  **Support** → 24/7 Bloomberg help desk staffed by financial
>     experts
>
> Free APIs work for academic projects. Bloomberg Terminal is what you
> use when managing real money.

### What You’ll Learn Today

By the end of this lab, you will have:

-   ✅ Used Bloomberg Desktop API (`blpapi`) to fetch professional-grade
    data
-   ✅ Compared Bloomberg data quality with yfinance (head-to-head)
-   ✅ Built a unified pipeline that handles both sources (with
    fallbacks)
-   ✅ Understood trade-offs: quality vs. cost, speed vs. access
-   ✅ Documented provenance (what data, from where, when, quality
    checks)

**Time estimate:** 120 minutes (in-lab only, requires Bloomberg Terminal
access)

> **Why This Matters for Your Career**
>
> If you want to work in trading, asset management, or investment
> banking, you’ll use Bloomberg daily. This lab gives you hands-on
> experience you can discuss in interviews: “I built data pipelines
> using Bloomberg API” is a concrete skill.

## Lab Objectives

By the end of this lab, you will:

1.  Use Bloomberg Desktop API (`blpapi`) to download professional-grade
    data
2.  Compare Bloomberg data quality with free APIs (yfinance)
3.  Build a pipeline that handles both data sources
4.  Understand trade-offs between professional and free data
5.  Document data provenance and quality differences

## Setup

### 1. Verify Bloomberg Terminal Access

Open Bloomberg Terminal and confirm you’re logged in. Test basic
functions: - Type `AAPL US Equity <GO>` - Type `GP <GO>` to see graphs -
Type `FLDS <GO>` to search for field mnemonics

### 2. Python Environment Check

In [1]:
import sys
import platform

print(f"Python: {sys.version}")
print(f"Platform: {platform.platform()}")

# Check for Bloomberg Educational Package
try:
    from bbg_edu import BloombergClient
    print("✓ bbg-edu package installed")
    
    # Check if Bloomberg Terminal is available
    client = BloombergClient()
    if client.is_available():
        print("✓ Bloomberg Terminal connected")
    else:
        print("⚠  Bloomberg Terminal not available")
        print("   Make sure Terminal is running and logged in")
except ImportError:
    print("✗ bbg-edu not installed")
    print("  Install with: pip install git+https://github.com/quinfer/bloomberg-edu.git")

# Check for other required packages
required = ['pandas', 'numpy', 'yfinance', 'matplotlib']
for pkg in required:
    try:
        __import__(pkg)
        print(f"✓ {pkg}")
    except ImportError:
        print(f"✗ {pkg} - install with: pip install {pkg}")

## Part 1: Bloomberg Data Pull

### Task 1.1: Initialize Bloomberg Client

We’ll use the `bbg-edu` educational package, which provides a clean
interface to Bloomberg’s Desktop API:

In [2]:
from bbg_edu import BloombergClient
import pandas as pd
from datetime import datetime, timedelta

# Initialize Bloomberg client
client = BloombergClient(
    validate_data=True,        # Automatic quality checks
    rate_limit_sec=2.0         # Conservative rate limiting
)

# Verify connection
if not client.is_available():
    raise RuntimeError("Bloomberg Terminal not available. Check it's running and logged in.")

print("✓ Bloomberg client initialized")
print("✓ Ready to fetch data")

### Task 1.2: Pull Historical Data for Core Equities

In [3]:
# Define our universe (start small for lab)
symbols_bbg = [
    "AAPL US Equity",
    "MSFT US Equity", 
    "GOOGL US Equity",
    "SPY US Equity"
]

fields_bbg = [
    "PX_OPEN",
    "PX_HIGH",
    "PX_LOW",
    "PX_LAST",
    "PX_VOLUME"
]

# Date range (last 2 years)
end_date = datetime.today()
start_date = end_date - timedelta(days=365*2)

print(f"Downloading {len(symbols_bbg)} symbols from Bloomberg")
print(f"Period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Fields: {', '.join(fields_bbg)}")

# Fetch historical data
bbg_data = client.historical(
    securities=symbols_bbg,
    fields=fields_bbg,
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d'),
    periodicity='DAILY'
)

print(f"\n✓ Downloaded {len(bbg_data)} rows")
print(f"  Date range: {bbg_data['date'].min()} to {bbg_data['date'].max()}")
print(f"  Tickers: {bbg_data['security'].nunique()}")

# View fetch metadata (provenance tracking)
print("\nFetch Metadata:")
for key, value in client.last_fetch_metadata.items():
    print(f"  {key}: {value}")

# Save to CSV for later comparison
bbg_data.to_csv('bloomberg_data.csv', index=False)
print("\n✓ Saved to bloomberg_data.csv")

### Task 1.3: Explore Bloomberg Data Quality

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# Load Bloomberg data
bbg = pd.read_csv('bloomberg_data.csv', parse_dates=['date'])

# Check data quality
print("Bloomberg Data Quality Report:")
print("=" * 50)

for ticker in bbg['security'].unique():
    ticker_data = bbg[bbg['security'] == ticker]
    
    # Count missing values
    missing = ticker_data[fields_bbg].isna().sum()
    
    # Check for data anomalies
    prices = ticker_data['PX_LAST']
    returns = prices.pct_change().dropna()
    extreme_returns = (returns.abs() > 0.2).sum()
    
    # Check High >= Low constraint
    if 'PX_HIGH' in ticker_data.columns and 'PX_LOW' in ticker_data.columns:
        high_low_violations = (ticker_data['PX_HIGH'] < ticker_data['PX_LOW']).sum()
    else:
        high_low_violations = 0
    
    print(f"\n{ticker}:")
    print(f"  Rows: {len(ticker_data)}")
    print(f"  Missing values: {missing.sum()} ({missing.sum()/len(ticker_data)*100:.1f}%)")
    print(f"  Extreme returns (>20%): {extreme_returns}")
    print(f"  High<Low violations: {high_low_violations}")

## Part 2: Free API Data Pull (Comparison)

### Task 2.1: Download Same Data from yfinance

In [5]:
import yfinance as yf
import time
import random

# Convert Bloomberg tickers to Yahoo format
symbols_yf = ['AAPL', 'MSFT', 'GOOGL', 'SPY']

def fetch_yfinance_with_retry(symbols, period='2y', max_tries=3):
    """
    Fetch historical data from Yahoo Finance with exponential backoff retry logic.
    
    Implements resilient API calls with automatic retries when rate limits or
    temporary failures occur. Uses exponential backoff to avoid overwhelming
    the API during high-traffic periods.
    
    Parameters
    ----------
    symbols : list of str
        Stock tickers in Yahoo Finance format (e.g., ['AAPL', 'MSFT'])
    period : str, default='2y'
        Historical period to fetch: '1d', '5d', '1mo', '3mo', '6mo', 
        '1y', '2y', '5y', '10y', 'ytd', 'max'
    max_tries : int, default=3
        Maximum number of retry attempts before raising error
        
    Returns
    -------
    pd.DataFrame
        Multi-level DataFrame with OHLCV data (Open, High, Low, Close, Volume)
        grouped by ticker. Index is datetime, columns are MultiIndex.
        
    Raises
    ------
    RuntimeError
        If all retry attempts fail
        
    Notes
    -----
    - Uses `auto_adjust=True` for split/dividend-adjusted prices
    - Implements exponential backoff: waits 1s, 2s, 4s between retries
    - Adds random jitter to avoid thundering herd
    - Returns only if non-empty DataFrame received
    
    Examples
    --------
    >>> data = fetch_yfinance_with_retry(['AAPL', 'MSFT'], period='1y')
    >>> data['AAPL']['Close'].iloc[-1]  # Latest AAPL closing price
    182.50
    >>> data.shape
    (252, 10)  # 252 days × (5 fields × 2 symbols)
    
    See Also
    --------
    yf.download : Underlying Yahoo Finance API
    get_close_from_yf : Alternative implementation from Lab 02 APIs
    """
    for attempt in range(max_tries):
        try:
            print(f"Attempt {attempt + 1}/{max_tries}...")
            data = yf.download(
                symbols, 
                period=period,
                auto_adjust=True,  # Adjust for splits/dividends
                progress=False,    # Suppress progress bar
                group_by='ticker'  # Group columns by ticker
            )
            
            if not data.empty:
                return data
                
        except Exception as e:
            print(f"  Error: {e}")
            if attempt < max_tries - 1:
                # Exponential backoff with random jitter
                sleep_time = 2 ** attempt + random.uniform(0, 1)
                print(f"  Retrying in {sleep_time:.1f}s...")
                time.sleep(sleep_time)
    
    raise RuntimeError(f"Failed after {max_tries} attempts")

# Download from yfinance
print("Downloading from Yahoo Finance...")
yf_data = fetch_yfinance_with_retry(symbols_yf)

# Restructure to match Bloomberg format
yf_clean = []
for ticker in symbols_yf:
    ticker_df = yf_data[ticker].copy()
    ticker_df['security'] = ticker
    ticker_df['date'] = ticker_df.index
    ticker_df = ticker_df.rename(columns={
        'Open': 'PX_OPEN',
        'High': 'PX_HIGH',
        'Low': 'PX_LOW',
        'Close': 'PX_LAST',
        'Volume': 'PX_VOLUME'
    })
    yf_clean.append(ticker_df[['date', 'security', 'PX_OPEN', 'PX_HIGH', 'PX_LOW', 'PX_LAST', 'PX_VOLUME']])

yf_combined = pd.concat(yf_clean, ignore_index=True)
yf_combined.to_csv('yfinance_data.csv', index=False)

print(f"\n✓ Downloaded {len(yf_combined)} rows from Yahoo Finance")
print(f"✓ Saved to yfinance_data.csv")

### Task 2.2: Yahoo Finance Data Quality

In [6]:
# Load Yahoo Finance data
yf_df = pd.read_csv('yfinance_data.csv', parse_dates=['date'])

print("Yahoo Finance Data Quality Report:")
print("=" * 50)

for ticker in yf_df['security'].unique():
    ticker_data = yf_df[yf_df['security'] == ticker]
    
    # Count missing values
    value_cols = ['PX_OPEN', 'PX_HIGH', 'PX_LOW', 'PX_LAST', 'PX_VOLUME']
    missing = ticker_data[value_cols].isna().sum()
    
    # Check for anomalies
    prices = ticker_data['PX_LAST']
    returns = prices.pct_change().dropna()
    extreme_returns = (returns.abs() > 0.2).sum()
    
    # High >= Low check
    high_low_violations = (ticker_data['PX_HIGH'] < ticker_data['PX_LOW']).sum()
    
    print(f"\n{ticker}:")
    print(f"  Rows: {len(ticker_data)}")
    print(f"  Missing values: {missing.sum()} ({missing.sum()/len(ticker_data)*100:.1f}%)")
    print(f"  Extreme returns (>20%): {extreme_returns}")
    print(f"  High<Low violations: {high_low_violations}")

## Part 3: Comparative Analysis

### Task 3.1: Direct Price Comparison

In [7]:
import matplotlib.pyplot as plt

# Load both datasets
bbg = pd.read_csv('bloomberg_data.csv', parse_dates=['date'])
yf_df = pd.read_csv('yfinance_data.csv', parse_dates=['date'])

# Compare AAPL prices
ticker_to_compare = 'AAPL'
bbg_ticker = 'AAPL US Equity'

bbg_aapl = bbg[bbg['security'] == bbg_ticker].set_index('date')['PX_LAST']
yf_aapl = yf_df[yf_df['security'] == ticker_to_compare].set_index('date')['PX_LAST']

# Merge on date
comparison = pd.DataFrame({
    'Bloomberg': bbg_aapl,
    'Yahoo_Finance': yf_aapl
}).dropna()

# Calculate differences
comparison['Difference'] = comparison['Bloomberg'] - comparison['Yahoo_Finance']
comparison['Pct_Difference'] = (comparison['Difference'] / comparison['Bloomberg']) * 100

print(f"Price Comparison for {ticker_to_compare}:")
print("=" * 50)
print(f"Overlapping dates: {len(comparison)}")
print(f"\nPrice Differences:")
print(f"  Mean difference: ${comparison['Difference'].mean():.4f}")
print(f"  Std dev: ${comparison['Difference'].std():.4f}")
print(f"  Max difference: ${comparison['Difference'].abs().max():.4f}")
print(f"  Mean % difference: {comparison['Pct_Difference'].mean():.6f}%")

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Plot prices
axes[0].plot(comparison.index, comparison['Bloomberg'], label='Bloomberg', alpha=0.7)
axes[0].plot(comparison.index, comparison['Yahoo_Finance'], label='Yahoo Finance', alpha=0.7)
axes[0].set_title(f'{ticker_to_compare} Price Comparison: Bloomberg vs. Yahoo Finance')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot difference
axes[1].plot(comparison.index, comparison['Difference'], color='red', alpha=0.7)
axes[1].axhline(0, color='black', linestyle='--', alpha=0.3)
axes[1].set_title('Price Difference (Bloomberg - Yahoo Finance)')
axes[1].set_ylabel('Difference ($)')
axes[1].set_xlabel('Date')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('bloomberg_vs_yfinance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Saved comparison chart to bloomberg_vs_yfinance_comparison.png")

### Task 3.2: Coverage and Completeness Analysis

In [8]:
# Analyze data coverage differences
print("Coverage Analysis:")
print("=" * 50)

for ticker_yf in symbols_yf:
    ticker_bbg = f"{ticker_yf} US Equity"
    
    # Bloomberg data
    bbg_ticker = bbg[bbg['security'] == ticker_bbg]
    bbg_dates = set(bbg_ticker['date'].dt.date)
    
    # Yahoo Finance data
    yf_ticker = yf_df[yf_df['security'] == ticker_yf]
    yf_dates = set(yf_ticker['date'].dt.date)
    
    # Find differences
    only_bbg = bbg_dates - yf_dates
    only_yf = yf_dates - bbg_dates
    overlap = bbg_dates & yf_dates
    
    print(f"\n{ticker_yf}:")
    print(f"  Bloomberg trading days: {len(bbg_dates)}")
    print(f"  Yahoo Finance trading days: {len(yf_dates)}")
    print(f"  Overlapping dates: {len(overlap)}")
    print(f"  Only in Bloomberg: {len(only_bbg)}")
    print(f"  Only in Yahoo Finance: {len(only_yf)}")
    
    if only_bbg:
        print(f"  Example Bloomberg-only dates: {sorted(list(only_bbg))[:3]}")
    if only_yf:
        print(f"  Example Yahoo-only dates: {sorted(list(only_yf))[:3]}")

## Part 4: Professional Pipeline with Both Sources

### Task 4.1: Create Unified Data Pipeline

In [9]:
import json
import hashlib
from datetime import datetime

class DataPipeline:
    """
    Professional-grade data pipeline supporting Bloomberg Terminal and free APIs.
    
    This class demonstrates production patterns: multi-source fetching, data
    validation, provenance logging, and integrity checking. Use this pattern
    for real trading/research systems where data quality is critical.
    
    Attributes
    ----------
    output_dir : str
        Directory for output files (data CSVs, metadata JSON)
    metadata : dict
        Comprehensive provenance log tracking all pipeline operations:
        - pipeline_version : version identifier
        - created : ISO timestamp of pipeline creation
        - python_version : Python version used
        - sources : list of fetch operations with details
        - validation results : quality checks per source
        - data_hash : SHA-256 hash for data integrity
        
    Examples
    --------
    >>> pipeline = DataPipeline(output_dir='my_data')
    >>> # Try Bloomberg first
    >>> data, source = pipeline.fetch_bloomberg(
    ...     symbols=["AAPL US Equity"],
    ...     fields=["PX_LAST"],
    ...     start_date='20230101',
    ...     end_date='20231231'
    ... )
    >>> if data is not None:
    ...     pipeline.validate(data, source)
    ...     pipeline.save(data, 'bloomberg_data.csv')
    >>> else:
    ...     # Fallback to free API
    ...     data, source = pipeline.fetch_yfinance(['AAPL'])
    ...     pipeline.save(data, 'fallback_data.csv')
    
    Notes
    -----
    The metadata JSON file provides complete reproducibility:
    - What data was fetched (symbols, fields, dates)
    - From where (Bloomberg vs yfinance)
    - When (timestamps for each operation)
    - Quality (validation results, issues found)
    - Integrity (data hash for change detection)
    
    This level of logging is required for:
    - Regulatory compliance (MiFID II, SEC Rule 15c3-5)
    - Research reproducibility (academic standards)
    - Production debugging (when models fail, audit the data)
    """
    
    def __init__(self, output_dir='pipeline_output'):
        """
        Initialize data pipeline with output directory and metadata tracking.
        
        Parameters
        ----------
        output_dir : str, default='pipeline_output'
            Directory path for all pipeline outputs (data CSVs, metadata JSON).
            Created automatically if it doesn't exist.
        """
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        self.metadata = {
            'pipeline_version': '1.0',
            'created': datetime.now().isoformat(),
            'python_version': sys.version.split()[0],
            'sources': []
        }
    
    def fetch_bloomberg(self, symbols, fields, start_date, end_date):
        """
        Fetch historical data from Bloomberg Terminal via Desktop API.
        
        Parameters
        ----------
        symbols : list of str
            Bloomberg security identifiers (e.g., ["AAPL US Equity", "MSFT US Equity"])
        fields : list of str
            Bloomberg field mnemonics (e.g., ["PX_LAST", "PX_VOLUME"])
            Use FLDS <GO> on Terminal to search available fields
        start_date : str
            Start date in YYYYMMDD format (e.g., '20230101')
        end_date : str
            End date in YYYYMMDD format (e.g., '20231231')
            
        Returns
        -------
        tuple of (pd.DataFrame, str) or (None, None)
            If successful: (data DataFrame, 'bloomberg')
            If failed: (None, None)
            
        Notes
        -----
        - Requires Bloomberg Terminal running and logged in
        - Requires `blpapi` Python package installed
        - Logs fetch metadata for provenance tracking
        - Fetches daily periodicity (can be modified for intraday)
        """
        print("Fetching from Bloomberg...")
        
        try:
            with BbgSession() as session:
                data = session.historical(
                    securities=symbols,
                    fields=fields,
                    start_date=start_date,
                    end_date=end_date,
                    periodicity='DAILY'
                )
            
            self.metadata['sources'].append({
                'provider': 'Bloomberg Desktop API',
                'symbols': symbols,
                'fields': fields,
                'date_range': [start_date, end_date],
                'rows': len(data),
                'fetch_time': datetime.now().isoformat()
            })
            
            return data, 'bloomberg'
            
        except Exception as e:
            print(f"Bloomberg fetch failed: {e}")
            return None, None
    
    def fetch_yfinance(self, symbols, period='2y'):
        """
        Fetch historical data from Yahoo Finance (fallback when Bloomberg unavailable).
        
        Parameters
        ----------
        symbols : list of str
            Yahoo Finance tickers (e.g., ['AAPL', 'MSFT'])
        period : str, default='2y'
            Historical period: '1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y'
            
        Returns
        -------
        tuple of (pd.DataFrame, str) or (None, None)
            If successful: (data DataFrame with standardized columns, 'yfinance')
            If failed: (None, None)
            
        Notes
        -----
        - Converts yfinance OHLCV format to match Bloomberg structure
        - Logs fetch metadata for provenance tracking
        - Uses retry logic from fetch_yfinance_with_retry()
        """
        print("Fetching from Yahoo Finance...")
        
        try:
            data = fetch_yfinance_with_retry(symbols, period)
            
            # Convert to standard format
            clean_data = []
            for sym in symbols:
                ticker_df = data[sym].copy()
                ticker_df['security'] = sym
                ticker_df['date'] = ticker_df.index
                clean_data.append(ticker_df)
            
            combined = pd.concat(clean_data, ignore_index=True)
            
            self.metadata['sources'].append({
                'provider': 'Yahoo Finance',
                'symbols': symbols,
                'period': period,
                'rows': len(combined),
                'fetch_time': datetime.now().isoformat()
            })
            
            return combined, 'yfinance'
            
        except Exception as e:
            print(f"Yahoo Finance fetch failed: {e}")
            return None, None
    
    def validate(self, data, source):
        """Validate data quality"""
        print(f"Validating {source} data...")
        
        issues = {
            'missing_values': int(data.isna().sum().sum()),
            'negative_prices': 0,
            'high_low_violations': 0,
            'extreme_returns': 0
        }
        
        # Check for negative prices
        price_cols = [c for c in data.columns if 'PX_' in c or 'Close' in c]
        for col in price_cols:
            if col in data.columns:
                issues['negative_prices'] += int((data[col] <= 0).sum())
        
        # Check High >= Low
        if 'PX_HIGH' in data.columns and 'PX_LOW' in data.columns:
            issues['high_low_violations'] = int((data['PX_HIGH'] < data['PX_LOW']).sum())
        
        self.metadata[f'{source}_validation'] = issues
        
        print(f"  Missing values: {issues['missing_values']}")
        print(f"  Negative prices: {issues['negative_prices']}")
        print(f"  High<Low violations: {issues['high_low_violations']}")
        
        return issues
    
    def save(self, data, filename):
        """Save data with metadata"""
        filepath = os.path.join(self.output_dir, filename)
        data.to_csv(filepath, index=False)
        
        # Generate data hash for integrity
        data_hash = hashlib.sha256(data.to_csv(index=False).encode()).hexdigest()[:12]
        self.metadata['data_hash'] = data_hash
        
        # Save metadata
        meta_path = os.path.join(self.output_dir, 'metadata.json')
        with open(meta_path, 'w') as f:
            json.dump(self.metadata, f, indent=2)
        
        print(f"✓ Saved data to {filepath}")
        print(f"✓ Saved metadata to {meta_path}")
        print(f"  Data hash: {data_hash}")

# Use the pipeline
pipeline = DataPipeline()

# Try Bloomberg first
bbg_symbols = ["AAPL US Equity", "MSFT US Equity"]
bbg_data, source = pipeline.fetch_bloomberg(
    symbols=bbg_symbols,
    fields=["PX_LAST", "PX_VOLUME"],
    start_date=(datetime.now() - timedelta(days=365)).strftime('%Y%m%d'),
    end_date=datetime.now().strftime('%Y%m%d')
)

if bbg_data is not None:
    issues = pipeline.validate(bbg_data, source)
    pipeline.save(bbg_data, 'professional_data.csv')
else:
    print("Bloomberg failed; would use yfinance fallback in production")

## Part 5: Lab Report

### Deliverables

Create a short report (1-2 pages) covering:

**1. Data Quality Comparison (30 points)** - Compare missing data rates
between Bloomberg and Yahoo Finance - Identify any price discrepancies
and explain why they occur - Assess data coverage differences

**2. Professional vs. Free API Trade-offs (25 points)** - What are the
advantages of Bloomberg data? - When would free APIs be sufficient? -
How do costs (time, money) factor into the decision?

**3. Pipeline Design (25 points)** - Explain your data validation
strategy - How did you handle errors and retries? - What metadata did
you log and why?

**4. Bloomberg Terminal Experience (20 points)** - What Bloomberg
functions did you explore? - How would you integrate Bloomberg into a
professional workflow? - What surprised you about Bloomberg vs. free
APIs?

### Report Template

``` markdown
# Lab 2 Report: Bloomberg Terminal Data Pipeline

**Name:** [Your Name]
**Date:** [Date]
**Lab Session:** [Session Time]

## 1. Data Quality Comparison

### Bloomberg Data
- Symbols analyzed: [list]
- Date range: [range]
- Missing data rate: [X%]
- Issues found: [describe]

### Yahoo Finance Data  
- Symbols analyzed: [list]
- Date range: [range]
- Missing data rate: [X%]
- Issues found: [describe]

### Price Discrepancies
[Describe any differences you found and explain why they occur]

## 2. Professional vs. Free API Trade-offs

### Bloomberg Advantages
- [List 3-5 advantages you observed]

### When Free APIs Are Sufficient
- [Discuss appropriate use cases]

### Cost-Benefit Analysis
- [Discuss time, money, and quality trade-offs]

## 3. Pipeline Design

### Validation Strategy
- [Explain your validation checks]

### Error Handling
- [Describe your retry logic and fallback strategy]

### Metadata Logging
- [List what you logged and why it matters]

## 4. Bloomberg Terminal Experience

### Functions Explored
- [List Bloomberg functions you tried: GP, FLDS, etc.]

### Professional Workflow Integration
- [How would you use Bloomberg in a real trading/research role?]

### Insights and Surprises
- [What did you learn? What surprised you?]

## Appendix: Code and Output
[Attach key code snippets and output screenshots]
```

## Additional Resources

### Bloomberg Terminal Functions to Explore

-   **FLDS <GO>**: Search for field mnemonics
-   **GP <GO>**: Graph prices with technical indicators
-   **FA <GO>**: Financial analysis (ratios, comps)
-   **ANR <GO>**: Analyst recommendations
-   **N <GO>**: News (real-time, searchable)
-   **CACS <GO>**: Corporate actions (splits, dividends)
-   **DES <GO>**: Security description
-   **HDS <GO>**: Historical data (alternative to API)

### Further Reading

-   Bloomberg API Documentation: Available on Bloomberg Terminal (DAPI
    <GO>)
-   Hilpisch (2019), Chapter 3: Financial Data (Bloomberg integration)
-   Ulster Bloomberg Training: Contact Financial Innovation Lab staff

### Troubleshooting

**Bloomberg API errors:** - Ensure Terminal is running and you’re logged
in - Check `blpapi` is installed: `pip list | grep blpapi` - Verify
security names: Use `FLDS <GO>` to check exact mnemonics

**Data discrepancies:** - Check adjustment settings (splits/dividends) -
Verify date ranges are comparable - Consider different market
hours/timezones

**Performance issues:** - Reduce batch sizes (max 6 tickers per request
recommended) - Increase sleep times between requests - Use time-limited
queries (max 2 years for lab)

> **Lab Hours and Support**
>
> -   Financial Innovation Lab hours: \[TBD\]
> -   Bloomberg Terminal bookings: Contact lab staff
> -   Technical support: Office hours or lab assistants